In [2]:
import librosa
import numpy as np
from IPython.display import Audio
import torch

In [8]:
from funasr import AutoModel

model = AutoModel(model="iic/emotion2vec_plus_large")

2024-06-12 23:49:17,668 - modelscope - INFO - PyTorch version 2.3.1 Found.
2024-06-12 23:49:17,672 - modelscope - INFO - Loading ast index from /home/mila/s/soroush.omranpour/.cache/modelscope/ast_indexer
2024-06-12 23:49:18,352 - modelscope - INFO - Loading done! Current index file version is 1.15.0, with md5 5eb56b46a2b5de5a5690e9f19ec1ca5d and a total number of 980 components indexed
2024-06-12 23:49:19,967 - modelscope - WARNING - Using the master branch is fragile, please use it with caution!
2024-06-12 23:49:19,969 - modelscope - INFO - Use user-specified model revision: master


Detect model requirements, begin to install it: /home/mila/s/soroush.omranpour/.cache/modelscope/hub/iic/emotion2vec_plus_large/requirements.txt
install model requirements successfully
ckpt: /home/mila/s/soroush.omranpour/.cache/modelscope/hub/iic/emotion2vec_plus_large/model.pt
init param, map: modality_encoders.AUDIO.extra_tokens from d2v_model.modality_encoders.AUDIO.extra_tokens in ckpt
init param, map: modality_encoders.AUDIO.alibi_scale from d2v_model.modality_encoders.AUDIO.alibi_scale in ckpt
init param, map: modality_encoders.AUDIO.local_encoder.conv_layers.0.0.weight from d2v_model.modality_encoders.AUDIO.local_encoder.conv_layers.0.0.weight in ckpt
init param, map: modality_encoders.AUDIO.local_encoder.conv_layers.0.2.1.weight from d2v_model.modality_encoders.AUDIO.local_encoder.conv_layers.0.2.1.weight in ckpt
init param, map: modality_encoders.AUDIO.local_encoder.conv_layers.0.2.1.bias from d2v_model.modality_encoders.AUDIO.local_encoder.conv_layers.0.2.1.bias in ckpt
init

In [2]:
from speechbrain.inference.interfaces import foreign_class


classifier = foreign_class(
    source="speechbrain/emotion-recognition-wav2vec2-IEMOCAP", 
    pymodule_file="custom_interface.py", 
    classname="CustomEncoderWav2vec2Classifier"
)

torchvision is not available - cannot save figures
/home/mila/s/soroush.omranpour/Projects/lucid_env/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/mila/s/soroush.omranpour/Projects/lucid_env/lib/python3.10/site-packages/transformers/configuration_utils.py:363: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0',

In [1]:
from transformers import Wav2Vec2ForSequenceClassification, AutoFeatureExtractor

feature_extractor = AutoFeatureExtractor.from_pretrained(
    'ehcalabres/wav2vec2-lg-xlsr-en-speech-emotion-recognition'
)
hf_model = Wav2Vec2ForSequenceClassification.from_pretrained(
    'ehcalabres/wav2vec2-lg-xlsr-en-speech-emotion-recognition'
)

/home/mila/s/soroush.omranpour/Projects/lucid_env/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/mila/s/soroush.omranpour/Projects/lucid_env/lib/python3.10/site-packages/transformers/configuration_utils.py:363: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Some weights of the model checkpoint at ehcalabres/wav2vec2-lg-xlsr-en-speech-emotion-recognition were not used when initializing Wav2Vec2ForSequenceClassification: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.output.bias', 'classifier

In [3]:
y, fs = librosa.load('LANDR- Meditation- Transcend 15 Mins-Balanced-Medium.mp3', sr=16000)
y.shape, fs

((14401202,), 16000)

In [5]:
import math
from tqdm.notebook import tqdm

classes = ['angry', 'happy', 'neutral', 'sad', 'unknown']

cache = {}
n = y.shape[0]
m = 5 * fs
k = math.ceil(n / m)

res = []
for i in tqdm(range(k)):
    # _, _, _, text_lab = classifier.classify_batch(torch.tensor(y[i*m:(i+1)*m]).unsqueeze(0))
    # res += text_lab
    
    # scores = model.generate(
    #     input=y[i*m:(i+1)*m], 
    #     is_final=(i == (k - 1)), 
    #     chunk_size=m,
    #     cache=cache,
    #     verbose=-1
    # )[0]['scores']
    # res += [classes[np.argmax(scores)]]

    inputs = feature_extractor(y[i*m:(i+1)*m], sampling_rate=fs, return_tensors="pt")
    with torch.no_grad():
        logits = hf_model(**inputs).logits
        pred = torch.argmax(logits, dim=-1).item()
        pred = hf_model.config.id2label[pred]
        res += [pred]

  0%|          | 0/181 [00:00<?, ?it/s]

In [6]:
res

['disgust',
 'disgust',
 'disgust',
 'disgust',
 'disgust',
 'disgust',
 'disgust',
 'disgust',
 'disgust',
 'disgust',
 'disgust',
 'disgust',
 'disgust',
 'disgust',
 'disgust',
 'disgust',
 'disgust',
 'disgust',
 'disgust',
 'disgust',
 'disgust',
 'disgust',
 'disgust',
 'disgust',
 'disgust',
 'disgust',
 'disgust',
 'disgust',
 'disgust',
 'disgust',
 'disgust',
 'disgust',
 'disgust',
 'disgust',
 'disgust',
 'disgust',
 'disgust',
 'disgust',
 'disgust',
 'disgust',
 'disgust',
 'disgust',
 'disgust',
 'disgust',
 'disgust',
 'neutral',
 'disgust',
 'disgust',
 'disgust',
 'disgust',
 'disgust',
 'disgust',
 'disgust',
 'disgust',
 'disgust',
 'disgust',
 'disgust',
 'disgust',
 'disgust',
 'disgust',
 'disgust',
 'disgust',
 'disgust',
 'disgust',
 'disgust',
 'disgust',
 'disgust',
 'disgust',
 'disgust',
 'disgust',
 'disgust',
 'disgust',
 'calm',
 'disgust',
 'disgust',
 'neutral',
 'disgust',
 'disgust',
 'disgust',
 'disgust',
 'disgust',
 'disgust',
 'disgust',
 'disgu